## Usage
This notebook takes a file, such as one with complex or phase information, creates a mask, and saves the results in `outpath`. 

### Parameters
`inpath` : str    
    The full path to the data file to create a mask from.

`outpath` : str    
    The full path to the output folder. 

`padding` : int   
    Amount to pad the mask by.

`min_size` : int    
    The smallest allowable object size (default 64)

`erosion_radius` : int   
    Radius of circle to erode the sample mask by (default 15)

`dilation_radius` : int    
    Radius of circle to dilate the sample mask by (default 5)


**The parameters should be provided by explicitly modifying the top cell content or using tools such as [papermill](https://papermill.readthedocs.io/en/latest/index.html). If the notebook is run as is, please define the parameters accordingly.**

### Dependencies
- numpy
- skimage

In [ ]:
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

from skimage import morphology, transform
from skimage.filters import threshold_multiotsu
from skimage.segmentation import inverse_gaussian_gradient

In [ ]:
with h5py.File(inpath, "r") as f:
    data = f["entry/data/data"][:]
image_stack = np.angle(data)

In [ ]:
# Flat disk shaped footprint, array of 0 and 1
neighbourhood_erode = morphology.disk(erosion_radius)  
neighbourhood_dilate = morphology.disk(dilation_radius)

## Initialise np array

In [ ]:
background_mask = np.empty(np.shape(image_stack), dtype=bool)
sample_mask = np.empty(np.shape(image_stack), dtype=bool)

## Mask creation

In [ ]:
from numpy._typing import NDArray

def get_processed_mask(
    image: NDArray,
    padding: int,
    min_size: int,
    neighbourhood_erode: NDArray | tuple,
    neighbourhood_dilate: NDArray | tuple,
) -> tuple[NDArray, NDArray]:
    """Return processed mask

    Parameters
    ----------
    image : NDArray
        The image data.
    padding : int
        Numerical amount to pad array.
    neighbourhood_erode : DArray|tuple
        Disk to erode the sample mask by 
    neighbourhood_dilate : DArray|tuple
        Disk to dilate sample mask by
    min_size : int
        The smallest allowable object size.

    Returns
    -------
    An NDArray of boolean mask values
    """
    full_back_mask = get_back_mask(image, min_size)
    sample_mask = ~full_back_mask
    eroded_sample_mask = morphology.binary_erosion(sample_mask, neighbourhood_erode)
    dilated_sample_mask = morphology.dilation(eroded_sample_mask, neighbourhood_dilate)
    padded_array = get_padded_array(dilated_sample_mask, padding)
    open_mask = ~dilated_sample_mask & padded_array
    back_section = dilated_sample_mask & padded_array
    return open_mask, back_section


def get_padded_array(array: NDArray, padding: int) -> NDArray:
    """Return padded array

    Parameters
    ----------
    array : NDArray
        Array to be padded.
    padding : int
        Numerical amount to pad array.

    Returns
    -------
    NDArray of boolean mask values
    """
    padded = np.zeros_like(array, dtype=bool)
    padded[padding:-padding, padding:-padding] = True
    return padded


def get_back_mask(image: NDArray, min_size: int) -> NDArray:
    """Process and return the background mask

    Parameters
    ----------
    image : NDArray
        The image data.
    min_size : int
        The smallest allowable object size.

    Returns
    -------
    An NDArray of boolean mask values
    """
    g_inv = inverse_gaussian_gradient(image)
    # separate pixels of input into separate classes
    thresholds = threshold_multiotsu(g_inv, classes=3)
    # indices of the bins to which each value in input array belongs
    regions = np.digitize(g_inv, bins=thresholds)
    # remove small connected components smaller than specified pixel size 'min_size'
    mask_back = morphology.remove_small_objects(regions == 2, min_size)
    return mask_back

## Process image stack

In [ ]:
for i in range(len(image_stack)):
    background_mask[i], sample_mask[i] = get_processed_mask(
        image_stack[i], padding, min_size, neighbourhood_erode, neighbourhood_dilate
    )

## Copy the original file and save to output filepath

In [ ]:
outpath = Path(inpath).parent if not outpath else Path(outpath)

In [ ]:
outfile_stem = Path(inpath).stem
outfile = outpath.joinpath(f"{outfile_stem}_mask.nxs")

In [ ]:
os.system(f"cp {inpath} {outfile}")
with h5py.File(outfile, "r+") as f:
    del f["entry/data/data"]
    f[f"entry/data/background_mask"] = background_mask    
    f[f"entry/data/sample_mask"] = sample_mask

## Figures

In [ ]:
start = 0
mid = np.rint(image_stack.shape[0]/2).astype(int)
end = image_stack.shape[0]-1

fig, (ax1, ax2, ax3) = plt.subplots(1,3, sharey=True)
ax1.set_title("First slice")
ax1.imshow(image_stack[start])
ax2.set_title("Mid slice")
ax2.imshow(image_stack[mid])
ax3.set_title("End slice")
ax3.imshow(image_stack[end])

plt.suptitle('Input data', fontsize=12)
plt.subplots_adjust(top=1.4)
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, sharey=True)
ax1.set_title("First slice")
ax1.imshow(background_mask[start])
ax2.set_title("Mid slice")
ax2.imshow(background_mask[mid])
ax3.set_title("End slice")
ax3.imshow(background_mask[end])

plt.suptitle('Background mask', fontsize=12)
plt.subplots_adjust(top=1.4)
plt.show()